<a href="https://colab.research.google.com/github/gehad-Ahmed30/Vegetable-classification-CNN-/blob/main/Vegetable_classification_and_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Dataset

In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("misrakahmed/vegetable-image-dataset")

print("Path to dataset files:", path)

Path to dataset files: /root/.cache/kagglehub/datasets/misrakahmed/vegetable-image-dataset/versions/1


# Import necessary libraries

In [2]:
import requests
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout,Input
from tensorflow.keras.callbacks import EarlyStopping
from google.colab import files
from tensorflow.keras.preprocessing import image


# Read Data

In [3]:
dataset_path=path
files=os.listdir(dataset_path)
print(files)

['Vegetable Images']


In [4]:
dataset_main_path=os.path.join(dataset_path,'Vegetable Images')
sub_folders=os.listdir(dataset_main_path)
print(sub_folders)

['test', 'train', 'validation']


# Load train, validation, and test datasets using ImageDataGenerator with normalization


In [5]:
train_dir = os.path.join(dataset_main_path, "train")
valid_dir = os.path.join(dataset_main_path, "validation")
test_dir = os.path.join(dataset_main_path, "test")
# Apply data augmentation and normalization
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    validation_split=0.2
)
val_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

# Load images from directories After Preprocessing

In [6]:
train_data=train_datagen.flow_from_directory(train_dir,target_size=(224,224),
                                       batch_size=32,
                                       class_mode='categorical',
                                             seed=42,
                                             shuffle=True)
val_data=val_datagen.flow_from_directory(valid_dir,target_size=(224,224),
                                       batch_size=32,
                                       class_mode='categorical',
                                         seed=42,
                                         shuffle=True)

Found 15000 images belonging to 15 classes.
Found 3000 images belonging to 15 classes.


## Build Model CNN

In [7]:
model=Sequential([
    Input(shape=(224,224,3)),
    Conv2D(32,(3,3),activation='relu'),
    MaxPooling2D(2,2),

    Conv2D(64,(3,3),activation='relu'),
    MaxPooling2D(2,2),

    Conv2D(128,(3,3),activation='relu'),
    MaxPooling2D(2,2),

    Flatten(),
    Dense(128,activation='relu'),
    Dropout(0.5),

    Dense(15,activation='softmax')
])

In [8]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 222, 222, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 111, 111, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 109, 109, 64)        │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 54, 54, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 52, 52, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 26, 26, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 86528)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │      11,075,712 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 15)                  │           1,935 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 11,170,895 (42.61 MB)

 Trainable params: 11,170,895 (42.61 MB)

 Non-trainable params: 0 (0.00 B)

# Fit Model

In [11]:
early_stop=EarlyStopping(monitor='val_loss',patience=3,restore_best_weights=True)


In [10]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.fit(train_data, epochs=20, batch_size=32, validation_data=val_data, callbacks=[early_stop])

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 197s 407ms/step - accuracy: 0.2728 - loss: 2.1795 - val_accuracy: 0.6733 - val_loss: 0.9915
Epoch 2/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 189s 402ms/step - accuracy: 0.6129 - loss: 1.1777 - val_accuracy: 0.7500 - val_loss: 0.7533
Epoch 3/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 185s 394ms/step - accuracy: 0.7138 - loss: 0.8664 - val_accuracy: 0.7950 - val_loss: 0.6682
Epoch 4/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 183s 391ms/step - accuracy: 0.7598 - loss: 0.7271 - val_accuracy: 0.8873 - val_loss: 0.3992
Epoch 5/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 183s 389ms/step - accuracy: 0.7996 - loss: 0.6128 - val_accuracy: 0.8807 - val_loss: 0.3703
Epoch 6/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 184s 391ms/step - accuracy: 0.8101 - loss: 0.5874 - val_accuracy: 0.9107 - val_loss: 0.3171
Epoch 7/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 209s 405ms/step - accuracy: 0.8312 - loss: 0.5187 - val_accuracy: 0.9210 - val_loss: 0.2590
Epoch 8/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 184s 391ms/step - accuracy: 0.8575 -